In [1]:
pip install scrapy

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
# Show Python version
import platform
platform.python_version()

'3.6.9'

In [3]:

import numpy as np
import pandas as pd
import re
import logging
import json
import scrapy
import re
import sys
import requests
from pandas.io.json import json_normalize 
from bs4 import BeautifulSoup
import os
import pickle
from pandas.io.json import json_normalize 
from scrapy.crawler import CrawlerProcess

/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [4]:
try:
    import scrapy
except:
    !pip install scrapy
    import scrapy
from scrapy.crawler import CrawlerProcess

In [5]:
import json
import logging

class JsonWriterPipeline(object):

    def open_spider(self, spider):
        self.file = open('quoteresult.jl', 'w')

    def close_spider(self, spider):
        self.file.close()

    def process_item(self, item, spider):
        line = json.dumps(dict(item)) + "\n"
        self.file.write(line)
        return item

In [6]:
import scrapy
import requests
# from selenium import webdriver
# from selenium.common.exceptions import NoSuchElementException


def clean(text):
    digits = [s for s in text if s.isdigit()]
    clean_text = ''.join(digits)
    if not clean_text:
        return None
    return int(clean_text)

class QuotesSpider(scrapy.Spider):
    name = "Wildberries"
    allowed_domains = ['wildberries.ru']
    start_urls = [
        'https://www.wildberries.ru/catalog/obuv/zhenskaya/sabo-i-myuli/myuli',

    ]

    custom_settings = {
        'LOG_LEVEL': logging.WARNING,
        'ITEM_PIPELINES': {'__main__.JsonWriterPipeline': 1}, # Used for pipeline 1
        'FEED_FORMAT':'json',                                 # Used for pipeline 2
        'FEED_URI': 'quoteresult.json'                        # Used for pipeline 2
    }

    # def parse(self, response):
    #     response.xpath("//div[@class='dtList i-dtList j-card-item ']/span[@class='text']").extract()
    #     for quote in response.css('.dtList i-dtList j-card-item '):
    #         time =  quote.css('div::attr(data-catalogercod1s)').extract()
    #         yield {'time':time,}

    def parse(self, response):
        for quote in response.css('.dtList-inner'):
            link = quote.css('.dtlist-inner-brand-name')
            title1 = link.css('.brand-name::text').get()
            title2 = link.css('.goods-name::text').get()
            title = title1 + '/' + title2

            catalogercod1s =  quote.css('div::attr(data-catalogercod1s)').extract()
            raw_price = quote.css('.lower-price::text').get()
            id = quote.css('div::attr(id)').extract()
            image = quote.css('img::attr(src)').extract()
            price =raw_price and clean(raw_price) or None
           
            link =  quote.css('a::attr(href)').extract()
            list_link =  quote.css('a::attr(href)')
            for parselink in list_link:
                character = parselink.css('.comments_reviews_link > span > i').extract()
            
        
            size = quote.xpath('//div[@class="quick-order"]/span[@class="sizes"]/a[@rel="nofollow"]/text()').extract()
            
            yield {
                'title': title,
                'price': price,
                'id' : id,
                'character': character,
                'timestamp':catalogercod1s, 
                'link ':link,
                'size': size,
                'image': image,

            }
            

            

In [7]:
process = CrawlerProcess({
    'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'
})

process.crawl(QuotesSpider)
process.start()

2020-12-07 16:59:04 [scrapy.utils.log] INFO: Scrapy 2.4.1 started (bot: scrapybot)
2020-12-07 16:59:04 [scrapy.utils.log] INFO: Versions: lxml 4.2.6.0, libxml2 2.9.8, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 20.3.0, Python 3.6.9 (default, Oct  8 2020, 12:12:24) - [GCC 8.4.0], pyOpenSSL 20.0.0 (OpenSSL 1.1.1h  22 Sep 2020), cryptography 3.2.1, Platform Linux-4.19.112+-x86_64-with-Ubuntu-18.04-bionic
2020-12-07 16:59:04 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.epollreactor.EPollReactor
2020-12-07 16:59:04 [scrapy.crawler] INFO: Overridden settings:
{'LOG_LEVEL': 30,
 'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'}
/usr/local/lib/python3.6/dist-packages/scrapy/extensions/feedexport.py:247: ScrapyDeprecationWarning: The `FEED_URI` and `FEED_FORMAT` settings have been deprecated in favor of the `FEEDS` setting. Please see the `FEEDS` setting docs for more details
  exporter = cls(crawler)


In [8]:
ll quoteresult.*

-rw-r--r-- 1 root 30133 Dec  7 16:59 quoteresult.jl
-rw-r--r-- 1 root 30235 Dec  7 16:59 quoteresult.json


In [9]:
!tail -n 2 quoteresult.jl

{"title": "Mascotte /\u041c\u044e\u043b\u0438", "price": 1634, "id": ["c7420065"], "character": [], "timestamp": [], "link ": ["/catalog/7420065/detail.aspx?targetUrl=GP"], "size": [], "image": [" //static.wbstatic.net/i/blank.gif", "//images.wbstatic.net/c252x336/new/7420000/7420065-1.jpg"]}
{"title": "Natali Collection /\u041c\u044e\u043b\u0438", "price": 2646, "id": ["c8156507"], "character": [], "timestamp": [], "link ": ["/catalog/8156507/detail.aspx?targetUrl=GP"], "size": [], "image": [" //static.wbstatic.net/i/blank.gif", "//images.wbstatic.net/c252x336/new/8150000/8156507-1.jpg"]}


In [10]:
!tail -n 2 quoteresult.json

{"title": "Natali Collection /\u041c\u044e\u043b\u0438", "price": 2646, "id": ["c8156507"], "character": [], "timestamp": [], "link ": ["/catalog/8156507/detail.aspx?targetUrl=GP"], "size": [], "image": [" //static.wbstatic.net/i/blank.gif", "//images.wbstatic.net/c252x336/new/8150000/8156507-1.jpg"]}
]